In [4]:
import sys

from geopy.geocoders import Nominatim
from pyspark import SparkContext
from pyspark.sql import *
from operator import add
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import RegexTokenizer
import pyspark.sql.functions as f
import pandas as pd
from pyspark.sql.functions import lower, col, size, length
import numpy as np

spark = SparkSession\
  .builder \
  .appName("Twitter_app") \
  .getOrCreate()

In [5]:
stopWords = [ 'rt', 'RT', 'it’s', 'don’t', 'can’t', 'I’m', 'able', 'about', 'above', 'abroad', 'according', 'accordingly', 'across', 'actually', 'adj', 'after', 'afterwards', 'again', 'against', 'ago', 'ahead', 'aint', 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'alongside', 'already', 'also', 'although', 'always', 'am', 'amid', 'amidst', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', 'arent', 'around', 'as', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'back', 'backward', 'backwards', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'begin', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'came', 'can', 'cannot', 'cant', 'cant', 'caption', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', 'cmon', 'co', 'co.', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', 'couldnt', 'course', 'cs', 'currently', 'dare', 'darent', 'definitely', 'described', 'despite', 'did', 'didnt', 'different', 'directly', 'do', 'does', 'doesnt', 'doing', 'done', 'dont', 'down', 'downwards', 'during', 'each', 'edu', 'eg', 'eight', 'eighty', 'either', 'else', 'elsewhere', 'end', 'ending', 'enough', 'entirely', 'especially', 'et', 'etc', 'even', 'ever', 'evermore', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'fairly', 'far', 'farther', 'few', 'fewer', 'fifth', 'first', 'five', 'followed', 'following', 'follows', 'for', 'forever', 'former', 'formerly', 'forth', 'forward', 'found', 'four', 'from', 'further', 'furthermore', 'get', 'gets', 'getting', 'given', 'gives', 'go', 'goes', 'going', 'gone', 'got', 'gotten', 'greetings', 'had', 'hadnt', 'half', 'happens', 'hardly', 'has', 'hasnt', 'have', 'havent', 'having', 'he', 'hed', 'hell', 'hello', 'help', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'heres', 'hereupon', 'hers', 'herself', 'hes', 'hi', 'him', 'himself', 'his', 'hither', 'hopefully', 'how', 'howbeit', 'however', 'hundred', 'id', 'ie', 'if', 'ignored', 'ill', 'im', 'immediate', 'in', 'inasmuch', 'inc', 'inc.', 'indeed', 'indicate', 'indicated', 'indicates', 'inner', 'inside', 'insofar', 'instead', 'into', 'inward', 'is', 'isnt', 'it', 'itd', 'itll', 'its', 'its', 'itself', 'ive', 'just', 'k', 'keep', 'keeps', 'kept', 'know', 'known', 'knows', 'last', 'lately', 'later', 'latter', 'latterly', 'least', 'less', 'lest', 'let', 'lets', 'like', 'liked', 'likely', 'likewise', 'little', 'look', 'looking', 'looks', 'low', 'lower', 'ltd', 'made', 'mainly', 'make', 'makes', 'many', 'may', 'maybe', 'maynt', 'me', 'mean', 'meantime', 'meanwhile', 'merely', 'might', 'mightnt', 'mine', 'minus', 'miss', 'more', 'moreover', 'most', 'mostly', 'mr', 'mrs', 'much', 'must', 'mustnt', 'my', 'myself', 'name', 'namely', 'nd', 'near', 'nearly', 'necessary', 'need', 'neednt', 'needs', 'neither', 'never', 'neverf', 'neverless', 'nevertheless', 'new', 'next', 'nine', 'ninety', 'no', 'nobody', 'non', 'none', 'nonetheless', 'noone', 'no-one', 'nor', 'normally', 'not', 'nothing', 'notwithstanding', 'novel', 'now', 'nowhere', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', 'ones', 'only', 'onto', 'opposite', 'or', 'other', 'others', 'otherwise', 'ought', 'oughtnt', 'our', 'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own', 'particular', 'particularly', 'past', 'per', 'perhaps', 'placed', 'please', 'plus', 'possible', 'presumably', 'probably', 'provided', 'provides', 'que', 'quite', 'qv', 'rather', 'rd', 're', 'really', 'reasonably', 'recent', 'recently', 'regarding', 'regardless', 'regards', 'relatively', 'respectively', 'right', 'round', 'said', 'same', 'saw', 'say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'seven', 'several', 'shall', 'shant', 'she', 'shed', 'shell', 'shes', 'should', 'shouldnt', 'since', 'six', 'so', 'some', 'somebody', 'someday', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specified', 'specify', 'specifying', 'still', 'sub', 'such', 'sup', 'sure', 'take', 'taken', 'taking', 'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', 'thatll', 'thats', 'thats', 'thatve', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'thered', 'therefore', 'therein', 'therell', 'therere', 'theres', 'theres', 'thereupon', 'thereve', 'these', 'they', 'theyd', 'theyll', 'theyre', 'theyve', 'thing', 'things', 'think', 'third', 'thirty', 'this', 'thorough', 'thoroughly', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'till', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'ts', 'twice', 'two', 'un', 'under', 'underneath', 'undoing', 'unfortunately', 'unless', 'unlike', 'unlikely', 'until', 'unto', 'up', 'upon', 'upwards', 'us', 'use', 'used', 'useful', 'uses', 'using', 'usually', 'v', 'value', 'various', 'versus', 'very', 'via', 'viz', 'vs', 'want', 'wants', 'was', 'wasnt', 'way', 'we', 'wed', 'welcome', 'well', 'well', 'went', 'were', 'were', 'werent', 'weve', 'what', 'whatever', 'whatll', 'whats', 'whatve', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'wheres', 'whereupon', 'wherever', 'whether', 'which', 'whichever', 'while', 'whilst', 'whither', 'who', 'whod', 'whoever', 'whole', 'wholl', 'whom', 'whomever', 'whos', 'whose', 'why', 'will', 'willing', 'wish', 'with', 'within', 'without', 'wonder', 'wont', 'would', 'wouldnt', 'yes', 'yet', 'you', 'youd', 'youll', 'your', 'youre', 'yours', 'yourself', 'yourselves', 'youve', 'zero', 'a', 'hows', 'i', 'whens', 'whys', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'uucp', 'w', 'x', 'y', 'z', 'I', 'www', 'amount', 'bill', 'bottom', 'call', 'computer', 'con', 'couldnt', 'cry', 'de', 'describe', 'detail', 'due', 'eleven', 'empty', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'forty', 'front', 'full', 'give', 'hasnt', 'herse', 'himse', 'interest', 'itse”', 'mill', 'move', 'myse”', 'part', 'put', 'show', 'side', 'sincere', 'sixty', 'system', 'ten', 'thick', 'thin', 'top', 'twelve', 'twenty', 'abst', 'accordance', 'act', 'added', 'adopted', 'affected', 'affecting', 'affects', 'ah', 'announce', 'anymore', 'apparently', 'approximately', 'aren', 'arent', 'arise', 'auth', 'beginning', 'beginnings', 'begins', 'biol', 'briefly', 'ca', 'date', 'ed', 'effect', 'et-al', 'ff', 'fix', 'gave', 'giving', 'heres', 'hes', 'hid', 'home', 'id', 'im', 'immediately', 'importance', 'important', 'index', 'information', 'invention', 'itd', 'keys', 'kg', 'km', 'largely', 'lets', 'line', 'll', 'means', 'mg', 'million', 'ml', 'mug', 'na', 'nay', 'necessarily', 'nos', 'noted', 'obtain', 'obtained', 'omitted', 'ord', 'owing', 'page', 'pages', 'poorly', 'possibly', 'potentially', 'pp', 'predominantly', 'present', 'previously', 'primarily', 'promptly', 'proud', 'quickly', 'ran', 'readily', 'ref', 'refs', 'related', 'research', 'resulted', 'resulting', 'results', 'run', 'sec', 'section', 'shed', 'shes', 'showed', 'shown', 'showns', 'shows', 'significant', 'significantly', 'similar', 'similarly', 'slightly', 'somethan', 'specifically', 'state', 'states', 'stop', 'strongly', 'substantially', 'successfully', 'sufficiently', 'suggest', 'thered', 'thereof', 'therere', 'thereto', 'theyd', 'theyre', 'thou', 'thoughh', 'thousand', 'throug', 'til', 'tip', 'ts', 'ups', 'usefully', 'usefulness', 've', 'vol', 'vols', 'wed', 'whats', 'wheres', 'whim', 'whod', 'whos', 'widely', 'words', 'world', 'youd', 'youre']

In [6]:
#Load the data

data = spark.read.format('json').options(header='true', inferSchema='true') \
  .load('./datasets/NoFilterEnglish2020-02-04.json')

### Exercise 1

In [109]:
data.filter(f.col('timestamp_ms').between(1580770795658, 1580770796665) ) \
    .select(f.regexp_replace(f.col('text'), r',|\n|;|:|(|)|@|\.|&|\\|\||-|_', '').alias('text'))\
    .withColumn('word', f.explode(f.split(f.col('text'), ' '))) \
    .groupBy('word') \
    .count() \
    .sort('count', ascending=False) \
    .show()

+------+-----+
|  word|count|
+------+-----+
|    RT|   20|
|   the|   11|
|     I|    7|
|    to|    7|
|    is|    6|
|     a|    5|
|   for|    4|
|  this|    4|
|    it|    4|
|    an|    4|
|    of|    3|
|      |    3|
|    at|    3|
|  make|    3|
|   you|    3|
|saying|    2|
|  your|    2|
|  need|    2|
|   You|    2|
|   and|    2|
+------+-----+
only showing top 20 rows



### Exercise 2

In [23]:
# Converting the text to the array of strings
words = data.filter(f.col('timestamp_ms').between(1580770795659, 1580770796665) ) \
    .select(f.regexp_replace(f.col('text'), r',|\.|&|:|;|\|@|\|-|_', '').alias('text')) \
    .withColumn('word', f.split(f.col('text'), ' ')).select('word')

# Removing the stopwords from the array of strings
sc = spark.sparkContext
broadcastVar = sc.broadcast(stopWords)
broadcastVar.value

remover = StopWordsRemover(inputCol="word", outputCol="filtered", stopWords=broadcastVar.value)
words = remover.transform(words)

# Strings to column + calculate most frequent
words.withColumn('filtered', f.explode('filtered')) \
    .groupBy('filtered') \
    .count() \
    .sort('count', ascending=False) \
    .show()

+---------------+-----+
|       filtered|count|
+---------------+-----+
|               |    3|
|            amp|    2|
|           Rush|    2|
|           fuck|    2|
|            hit|    1|
|       clinging|    1|
|           Nice|    1|
|          rolls|    1|
|            (ie|    1|
|      @yungcalc|    1|
|@washingtonpost|    1|
|        explain|    1|
|       jaCCuZZi|    1|
|             i…|    1|
|  @iBeSuckaFree|    1|
|           icon|    1|
|         hyolyn|    1|
|      @dbongino|    1|
|          Trump|    1|
|           rare|    1|
+---------------+-----+
only showing top 20 rows



### Exercise 3

In [161]:
# Converting the text to the array of strings
words = data.filter("(timestamp_ms / 1000 / 60 / 60  % 24 )>= 20") \
    .select(f.regexp_replace(f.col('text'), r',|\.|’|\'|&|:|;|\|@|\||-|_', '').alias('text')) \
    .withColumn('word', f.split(f.col('text'), ' ')).select('word')

# Removing the stopwords from the array of strings
remover = StopWordsRemover(inputCol="word", outputCol="filtered", stopWords=broadcastVar.value)
words = remover.transform(words)

# Strings to column + calculate most frequent
words.withColumn('filtered', f.explode('filtered')) \
    .groupBy('filtered') \
    .count() \
    .sort('count', ascending=False) \
    .show()

+----------------+-----+
|        filtered|count|
+----------------+-----+
|                |58085|
|             amp| 6585|
|          people| 5388|
|            love| 4842|
|            time| 4108|
|             day| 3357|
|            good| 3084|
|           Trump| 3008|
|            Iowa| 2974|
|            shit| 2692|
|             man| 2428|
|           today| 2375|
|            yall| 2209|
|               2| 1976|
|           years| 1943|
|@realDonaldTrump| 1819|
|            life| 1805|
|            feel| 1700|
|            work| 1658|
|            fuck| 1607|
+----------------+-----+
only showing top 20 rows



### Exercise 5

In [5]:
hashtags = data.select(f.regexp_replace(f.col('text'), r',|\n|;|:|(|)|@|\.|&|\\|\||-|_', '').alias('text'))\
    .withColumn('hashtag', f.explode(f.split(f.col('text'), ' ')))

hashtags = hashtags.filter( hashtags.hashtag.startswith('#') )

In [6]:
most_freq_hash = hashtags.groupBy('hashtag') \
    .count() \
    .sort('count', ascending=False) 
most_freq_hash.show()

+--------------------+-----+
|             hashtag|count|
+--------------------+-----+
|       #IowaCaucuses| 2438|
|#DeshKiAwaazShehnaaz| 1741|
|       #iHeartAwards| 1421|
|    #MainBhiAsimKiGF| 1400|
|      #MeraAdarshSid| 1394|
|                  #1| 1146|
|                  #…| 1118|
|           #AsimRiaz| 1099|
|        #TheBachelor| 1068|
|                #BTS| 1020|
|           #IACaucus|  980|
|         #IowaCaucus|  973|
|     #SidharthShukla|  917|
|               #BB13|  716|
|        #coronavirus|  709|
|#CureCancer_By_Tr...|  683|
|         #LoveIsland|  646|
|           #TREASURE|  625|
|          #SuperBowl|  610|
| #TREASURE_EDITORIAL|  576|
+--------------------+-----+
only showing top 20 rows



In [7]:
mfht = most_freq_hash.filter('count >= 500').select('hashtag').rdd.map(list)

### Fucntion for location 

In [8]:
def loc(coords):

    geolocator = Nominatim(user_agent="geoapiExercises")  
    if coords.shape != (2,):
        coords = coords.mean(axis = 0)
    
    Longitude = str(coords[0])
    Latitude = str(coords[1])
    
    location = geolocator.reverse(Latitude+","+Longitude)
    
    if(location == None):
        print("No location specified")
        return
 
    address = location.raw['address']

    # traverse the data
    city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    code = address.get('country_code')
    zipcode = address.get('postcode')
    print('City : ', city)
    print('State : ', state)
    print('Country : ', country)
    print('Zip Code : ', zipcode)

### Exercise 6 and Exercice 7

In [9]:
for row in mfht.collect():
    ht = row[0]
    # Selecting the tweets for current Event
    event_words = data.filter( f.col('text').contains(ht))
    
    # Ex.6 : Finding timeframe of the Event
    end = event_words.select(f.max(f.col('timestamp_ms'))).rdd.map(list).collect()[0][0]
    start = event_words.select(f.min(f.col('timestamp_ms'))).rdd.map(list).collect()[0][0]
    end = str( pd.to_datetime(end, unit='ms').to_pydatetime())
    start = str(pd.to_datetime(start, unit='ms').to_pydatetime())
    
    # Ex.7 : Finding location of the Event
    coords = event_words.na.drop(subset=["place"]) \
    .select("place.bounding_box.coordinates") \
    .rdd.map(list).collect()
    
    # Printing the results
    print("Event : "+ht)
    print("Timeframe : from - "+start[:-7] + " to - " + end[:-7])
    if(coords != []):
        coords_mean = np.array(coords).squeeze().mean(axis = 0)
        loc(coords_mean)
    else: 
        print("No location specified")
    print('\n')

Event : #IowaCaucuses
Timeframe : from - 2020-02-03 23:00:11 to - 2020-02-04 22:58:28
City :  
State :  West Virginia
Country :  United States
Zip Code :  None




KeyboardInterrupt: 

### Benchmarking of different methods for searching minumum and maximum Timestamp 

In [127]:
import time

# Method 1: Use describe()
t1 = time.time()
float(event_words.describe("timestamp_ms").filter("summary = 'max'").select("timestamp_ms").collect()[0].asDict()['timestamp_ms'])

t2 = time.time()
print("Method 1")
print("time spent computing: {:.4g}".format(t2-t1))

# Method 2: Use SQL
t1 = time.time()

event_words.registerTempTable("df_table")
spark.sql("SELECT MAX(timestamp_ms) as maxval FROM df_table").collect()[0].asDict()['maxval']

t2 = time.time()
print("Method 2")
print("time spent computing: {:.4g}".format(t2-t1))

# Method 3: Convert to RDD
t1 = time.time()

event_words.select("timestamp_ms").rdd.max()[0]

t2 = time.time()
print("Method 3")
print("time spent computing: {:.4g}".format(t2-t1))


# Method 4: select
t1 = time.time()

event_words.select(f.max(f.col('timestamp_ms')))

t2 = time.time()
print("Method 4")
print("time spent computing: {:.4g}".format(t2-t1))

Method 1
time spent computing: 1.699
Method 2
time spent computing: 1.634
Method 3
time spent computing: 1.692
Method 4
time spent computing: 0.004184


### Exercice 8

In [9]:
event_words = data.filter( f.col('text').contains('#IowaCaucuses'))

In [57]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import col
from pyspark.sql import Row

from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier


from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

event_data = event_words.select('id','text','possibly_sensitive')

# set seed for reproducibility
(trainingData, testData) = event_data.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))
#
## regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
#
## stop words
add_stopwords = ["http","https","amp","rt","t","c","the","@"]
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
#
## bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
#
## convert string labels to indexes#
#print (label_stringIdx)

rf = RandomForestClassifier( featuresCol="features", numTrees=100, seed=100)
##lrModel = lr.fit(trainingData)
#
#
## build the pipeline
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, rf])
#pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, lr])


#
## Fit the pipeline to training documents.
predictions = pipeline.transform(testData)

predictions.select("text","possibly_sensitive","probability","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 30, truncate = 40)


# Evaluate, metricName=[accuracy | f1]default f1 measure
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
print("Accuracy: %g" % (evaluator.evaluate(predictions)))

# save the trained model for future use
pipelineFit.save("logreg.model")



Training Dataset Count: 2258
Test Dataset Count: 946


AttributeError: 'Pipeline' object has no attribute 'transform'

In [12]:
import sparknlp

from pyspark.ml import Pipeline
from sparknlp.base import *

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")
tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("token")
normalizer = Normalizer()\
    .setInputCols(["token"])\
    .setOutputCol("normal")
word_embeddings=WordEmbeddingsModel.pretrained()\
    .setInputCols(["document","normal"])\
    .setOutputCol("embeddings")
nlpPipeline = Pipeline(stages=[
    document_assembler, 
    sentenceDetector,
    tokenizer,
    normalizer,
    word_embeddings,
 ])
pipelineModel = nlpPipeline.fit(event_words)

TypeError: 'JavaPackage' object is not callable

SyntaxError: invalid syntax (<ipython-input-15-c264bcd168e6>, line 1)